In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

results = pd.DataFrame(index=['PacketsSent', 'PacketsSentFLC', 'PacketsReceived', 'PacketsReceivedFLC', 
                              'PacketsLost(%)', 'PacketsLostFLC(%)', 'Jitter', 'JitterFLC', 'Latency', 'LatencyFLC',
                              'PowerConsumed(W)', 'PowerConsumedFLC(W)'])

wsn1 = pd.read_csv('wsn1.csv', usecols=['run', 'type', 'module', 'name', 'value', 'mean'])

run = "wsnSc1T"

for i in range(1, 11):
    packetSentCount = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='scalar') & 
                             (wsn1.module.str.startswith("Sc1.sensor")) & (wsn1.module.str.endswith("udp")) & 
                             (wsn1.name=='packetSent:count')]   

    packetReceivedCount = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1'))& (wsn1.type=='scalar') & 
                                 (wsn1.module.str.startswith("Sc1.sink")) & 
                                 (wsn1.module.str.endswith("udp")) & (wsn1.name=='packetReceived:count')]

    totalPkSentCount = sum(packetSentCount.value)
    totalPkReceivedCount = sum(packetReceivedCount.value)
    packetLoss = (totalPkSentCount - totalPkReceivedCount) * 100 / totalPkSentCount

    residualEnergy = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='scalar') & 
                            (wsn1.name=='residualEnergyCapacity:last') & 
                            (wsn1.module.str.endswith('energyStorage'))] 

    spentEnergyJ = 3000 - sum(residualEnergy.value)
    consumedPower = spentEnergyJ / 240

    latency = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='histogram') & 
                     (wsn1.name=='endToEndDelay:histogram')] 

    latencyMean = latency["mean"].mean()
    
    jitter = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-1')) & (wsn1.type=='histogram') & 
                    (wsn1.name=='jitter:histogram')] 
    
    jitterMean = jitter["mean"].mean()

                        ########################################################
                        ########Simulation with FUZZY LOGIC CONTROLLER##########
                        ########################################################
    
    packetSentCountFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & (wsn1.type=='scalar') & 
                                (wsn1.module.str.startswith("Sc1.sensor")) & 
                                (wsn1.module.str.endswith("udp")) & (wsn1.name=='packetSent:count')] 

    packetReceivedCountFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0'))& (wsn1.type=='scalar') & 
                                    (wsn1.module.str.startswith("Sc1.sink")) & 
                                    (wsn1.module.str.endswith("udp")) & (wsn1.name=='packetReceived:count')]

    totalPkSentCountFLC = sum(packetSentCountFLC.value)
    totalPkReceivedCountFLC = sum(packetReceivedCountFLC.value)
    packetLossFLC = (totalPkSentCountFLC - totalPkReceivedCountFLC) * 100 / totalPkSentCountFLC

    residualEnergyFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & (wsn1.type=='scalar') & 
                               (wsn1.name=='residualEnergyCapacity:last') & 
                               (wsn1.module.str.endswith('energyStorage'))] 

    spentEnergyJFLC = 3000 - sum(residualEnergyFLC.value)
    consumedPowerFLC = spentEnergyJFLC / 240

    latencyFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & (wsn1.type=='histogram') & 
                        (wsn1.name=='endToEndDelay:histogram')] 

    latencyMeanFLC = latencyFLC["mean"].mean()

    jitterFLC = wsn1[(wsn1.run.str.startswith('wsnSc1T'+str(i)+'-0')) & 
                       (wsn1.type=='histogram') & (wsn1.name=='jitter:histogram')] 

    jitterMeanFLC = jitterFLC["mean"].mean()   
    
                        ########################################################
                        ########                 RESULTS             ##########
                        ########################################################
            
    runDataCollector = [totalPkSentCount, totalPkSentCountFLC, totalPkReceivedCount, totalPkReceivedCountFLC,
                        packetLoss, packetLossFLC, jitterMean, jitterMeanFLC, latencyMean, latencyMeanFLC, 
                        consumedPower, consumedPowerFLC]
    results.insert(i-1, "T"+str(i), runDataCollector)
        
print(results)


                             T1          T2           T3           T4  \
PacketsSent          798.000000  865.000000  2618.000000  3355.000000   
PacketsSentFLC       853.000000  953.000000  2558.000000  3575.000000   
PacketsReceived      786.000000  848.000000  2578.000000  3324.000000   
PacketsReceivedFLC   846.000000  940.000000  2519.000000  3514.000000   
PacketsLost(%)         1.503759    1.965318     1.527884     0.923994   
PacketsLostFLC(%)      0.820633    1.364113     1.524629     1.706294   
Jitter                 0.009548    0.014102     0.017576     0.017438   
JitterFLC              0.006057    0.017779     0.026271     0.026329   
Latency                0.021596    0.044083     0.107602     0.138513   
LatencyFLC             0.025684    0.046741     0.094509     0.152082   
PowerConsumed(W)       1.529416    1.559726     1.657103     1.696323   
PowerConsumedFLC(W)    1.020330    1.045975     1.100126     1.141960   

                              T5            T6    